#### SE PIDE:
A partir del fichero ventas.csv diseña un modelo de base de datos relacional, almacénalo en
una base de datos SQLite y crea una tabla en la que se asocie cada cliente con su idioma.
Para ello:
1) Diseña el diagrama de la base de datos. Puedes elegir el nombre que prefieras para cada una de las tablas. Utiliza cualquier programa de edición de imágenes o la herramienta web https://dbdiagram.io/d

2) Crea una base de datos llamada karls.sqlite que contenga las tablas del modelo relacional con los datos correspondientes.

3) Accede a la base de datos world y construye la tabla que relacione cada cliente con su idioma. Añade esta tabla a la base de datos del paso anterior con el nombre deidiomas_cliente


In [1]:
# PEGAR png del diagrama

In [52]:
import pandas as pd
import sqlite3

df = pd.read_csv('./files/ventas.csv')

# Columnas para la tabla de productos
cols_productos = ['codigo_producto','precio_compra',
                 'nombre_producto','linea_productos', 'descr']

# Columnas para la tabla de ventas
cols_ventas = ['no_orden','codigo_producto','cantidad_orden',
               'precio_cada','fecha_orden','no_cliente']

# Columnas para la tabla de clientes
cols_clientes = ['no_cliente','nombre_cliente','tlf',
                 'direccion','estado','codigo_postal',
                 'pais']

# Creamos la base de datos sqlite
karls = sqlite3.connect('karls.sqlite')
cursor = karls.cursor()

# Incluimos las tablas en labase de datos
df[cols_productos].drop_duplicates().to_sql('productos', karls, index='False', if_exists='replace')
df[cols_ventas].drop_duplicates().to_sql('ventas', karls, index='False', if_exists='replace')
df[cols_clientes].drop_duplicates().to_sql('clientes', karls, index='False', if_exists='replace')

In [116]:
# Nos conectamos a la base de datos world para construir la tabla idiomaCliente
import pymysql
database_host = 'relational.fit.cvut.cz'
username = 'guest'
password = 'relational'
database_name = 'world'

# Abrir conexión con la base de datos
db = pymysql.connect(host=database_host, user=username, password=password, database=database_name)

query = '''
SELECT
    c.Name as pais,
    cl.Language as idioma,
    cl.IsOfficial,
    cl.Percentage
FROM Country c
JOIN CountryLanguage cl
ON c.Code = cl.CountryCode'''
pd.read_sql(query, db).head()

,pais,idioma,IsOfficial,Percentage
0,Aruba,Dutch,T,5.3
1,Aruba,English,F,9.5
2,Aruba,Papiamento,F,76.7
3,Aruba,Spanish,F,7.4
4,Afghanistan,Balochi,F,0.9


In [125]:
# Se observa que un mismo país puede tener varios idiomas.
# Se decide usar el idioma con mayor porcentaje, independientemente de que sea oficial o no.
query = '''
SELECT
    cl.CountryCode,
    c.Name as pais,
    cl.Language as idioma,
    cl.Percentage
FROM Country c
JOIN CountryLanguage cl
ON c.Code = cl.CountryCode
WHERE (cl.CountryCode, cl.Percentage) IN
    (SELECT CountryCode, MAX(Percentage) as Percentage
     FROM CountryLanguage
     GROUP BY CountryCode)'''

temp = pd.read_sql(query, db)
temp.head()
print (len(temp))
print (len(temp[['pais']].drop_duplicates()))
temp.head()

241
233


,CountryCode,pais,idioma,Percentage
0,ABW,Aruba,Papiamento,76.7
1,AFG,Afghanistan,Pashto,52.4
2,AGO,Angola,Ovimbundu,37.2
3,AIA,Anguilla,English,0.0
4,ALB,Albania,Albaniana,97.9


In [126]:
# Parece que todavía hay países que aparecen dos veces o más
# Vamos a tratar de ver a que se debe

# ¿Qué paises son?

paises_2_id = list(temp.groupby(['CountryCode','pais'], as_index=0).count().sort_values('idioma', ascending=False).head(8)['CountryCode'])
temp.groupby(['CountryCode','pais'], as_index=0).count().sort_values('idioma', ascending=False).head(8)


,CountryCode,pais,idioma,Percentage
152,NIU,Niue,2,2
203,TKL,Tokelau,2,2
35,CCK,Cocos (Keeling) Islands,2,2
205,TMP,East Timor,2,2
43,COK,Cook Islands,2,2
49,CXR,Christmas Island,2,2
226,WLF,Wallis and Futuna,2,2
183,SJM,Svalbard and Jan Mayen,2,2


In [129]:
# Buscamos a estos paises en la tabla CountryLanguage para saber su porcentajes de idiomas
query = '''
SELECT
    *
FROM CountryLanguage'''
temp = pd.read_sql(query, db)
temp[temp['CountryCode'].isin(paises_2_id)]

,CountryCode,Language,IsOfficial,Percentage
149,CCK,English,T,0.0
150,CCK,Malay,F,0.0
200,COK,English,F,0.0
201,COK,Maori,T,0.0
219,CXR,Chinese,F,0.0
220,CXR,English,T,0.0
655,NIU,English,T,0.0
656,NIU,Niue,F,0.0
782,SJM,Norwegian,T,0.0
783,SJM,Russian,F,0.0


In [134]:
# Todos esto paises tienen 0 en cualquier idioma
# Por tanto, se decide escoger el idioma oficial para estos paises

query = '''
SELECT
    cl.CountryCode,
    c.Name as pais,
    cl.Language as idioma,
    cl.Percentage
FROM Country c
JOIN CountryLanguage cl
ON c.Code = cl.CountryCode
WHERE (cl.CountryCode, cl.Percentage) IN
    (SELECT CountryCode, MAX(Percentage) as Percentage
     FROM CountryLanguage
     GROUP BY CountryCode)'''

idiomas_paises = pd.read_sql(query, db)
# Excluimos los paises repetidos
idiomas_paises = idiomas_paises[~idiomas_paises['CountryCode'].isin(paises_2_id)]

# realizamos otra query para quedarnos conel idioma oficial de los paises escluidos
query = '''
SELECT
    cl.CountryCode,
    c.Name as pais,
    cl.Language as idioma,
    cl.Percentage
FROM Country c
JOIN CountryLanguage cl
ON c.Code = cl.CountryCode
WHERE cl.IsOfficial = "T"'''
excluidos = pd.read_sql(query,db)
excluidos = excluidos[excluidos['CountryCode'].isin(paises_2_id)]

# Los concatenamos a la tabla de idiomas paises
idiomas_paises = pd.concat([idiomas_paises, excluidos])

# Comprobamos que ahora ya no hay paises duplicados
print (len(idiomas_paises))
print (len(idiomas_paises[['pais']].drop_duplicates()))

232
232


In [136]:
# Nos quedamos con las columnas necesarias y agregamos la tabla a la bases de datos sqlite
idiomas_paises[['pais','idioma']].to_sql('idiomaCliente', karls, index='False', if_exists='replace')

In [138]:
# Tablas de la base de datos
pd.read_sql('SELECT name FROM sqlite_master WHERE type="table"', karls)

,name
0,productos
1,ventas
2,clientes
3,idiomaCliente


##### OPCIONAL: Realiza una consulta SQL para conocer el producto que más beneficios genera.

In [144]:
query = '''
SELECT
    p.nombre_producto,
    v.imp_total_venta - (v.cantidad_vendida*p.precio_compra) as margen_total_producto
FROM (SELECT
          codigo_producto,
          sum(cantidad_orden) as cantidad_vendida,
          sum(cantidad_orden*precio_cada) as imp_total_venta
      FROM ventas
      GROUP BY codigo_producto) v
JOIN productos p
ON v.codigo_producto = p.codigo_producto
ORDER BY margen_total_producto DESC'''


pd.read_sql(query, karls)

,nombre_producto,margen_total_producto
0,1992 Ferrari 360 Spider red,135996.78
1,1952 Alpine Renault 1300,95282.58
2,2001 Ferrari Enzo,93349.65
3,2003 Harley-Davidson Eagle Drag Bike,81031.30
4,1968 Ford Mustang,72579.26
5,1969 Ford Falcon,72399.77
6,1928 Mercedes-Benz SSK,68423.18
7,2002 Suzuki XREO,67641.47
8,1980s Black Hawk Helicopter,64599.11
9,1948 Porsche Type 356 Roadster,62725.78
